# Tender2Project
The following Kaggle notebook exploits the long context window of Gemini, in order to fulfill the following targets:
- Analyze the tender for a project.
- Analyze the information about possible products, provided from different companies.
- Find the best combination of products to build the project in the tender, identifying the most compliant company as well.

## Notebook structure
The notebook is composed by different parts, each one with a specific target:
- A tender for a project is parsed, such that its information is converted to text.
- Information scraped from the websites of different companies is loaded as text.
- All the text is forwarded to Gemini, whereas a system prompt and a user prompt are written to explain the purposed o Gemini.

## Theoretical aspects
The current way to use Gemini makes use of the following properties of a LLM (Large Language Model) like Gemini:
|  **LLM property** | **Where it is used** | **How it is used** |
|:-----------------:|:--------------------:|:------------------:|
|     Reasoning     |          TBD         |         TBD        |
|       Memory      |          TBD         |         TBD        |
| Chain of Thoughts |          TBD         |         TBD        |
|                   |                      |                    |

Clean the working directory

In [1]:
! rm -r /kaggle/working/*

In [2]:
from IPython.display import Markdown

# Load the tenders

Locate the tenders - in PDF format.

In [3]:
# fetch the script to download content from GitHub
!wget https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/github_downloader.py -P /kaggle/working/scripts

# add downloaded script to the Python path
import sys
sys.path.append('/kaggle/working/scripts')

--2024-11-23 10:32:52--  https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/github_downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1722 (1.7K) [text/plain]
Saving to: '/kaggle/working/scripts/github_downloader.py'

github_downloader.p 100%[===================>]   1.68K  --.-KB/s    in 0s      

2024-11-23 10:32:52 (26.7 MB/s) - '/kaggle/working/scripts/github_downloader.py' saved [1722/1722]



In [4]:
import github_downloader

github_downloader.download_files_from_github_repo(folderName="tenders", saveFolder="/kaggle/working/tenders", extension="pdf")

/kaggle/working/tenders/tender_solar.pdf downloaded successfully.
/kaggle/working/tenders/tender_wind.pdf downloaded successfully.
All files downloaded.


In [5]:
import os

# if the PDF file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_tender = False if os.path.exists('/kaggle/working/tenders') else True
if use_kaggle_input_tender:
    tenders_file_path = '/kaggle/input/tenders'
else:
    tenders_file_path = '/kaggle/working/tenders'

print(f"The folder {tenders_file_path} will be considered as containing the tenders")

The folder /kaggle/working/tenders will be considered as containing the tenders


Installing required Python packages to analyze the tender - in PDF format.

In [6]:
!pip install PyPDF2

Extract information from the tenders.
The output will be a text.

In [7]:
import os
from PyPDF2 import PdfReader

tenders = [t for t in os.listdir(tenders_file_path) if t.endswith(".pdf")]
tenders_info = {}
for tender in tenders:
    print(f"Reading the tender {tender} ...")
    reader = PdfReader(os.path.join(tenders_file_path, tender))

    tenders_info[tender] = {}
    tenders_info[tender]["name"] = tender
    tenders_info[tender]["content"] = "\n".join([page.extract_text() for page in reader.pages])

# information for each tender can be accessed by:
# tenders_info[tenders[0]]

Reading the tender tender_solar.pdf ...
Reading the tender tender_wind.pdf ...


# Fetch information about companies

## Overview
Information about interesting companies is obtained from their websites.

To generate data out of the companies' websites, we implemented a crawler.
The final output of the crawler is a JSON file, in which each field refers to a company: for each company, all the information of the websites is merged.

> To make things easier, the mentioned JSON file will be fetched from a Git repository where the crawling function has already been executed.

## Details about the crawling process:
- **Recursive scan**: after a webpage is scanned and its content is stored, eventual found sublinks are scanned, as well. A limit of the wepages to download is given as input.
- **Redundant information is deleted**: if some website content can be found multiple times in all the webpages of one company, then it is skipped. *Example*: undesired and redundant lines like "Contact Us" are removed, ensuring that the final content does not include unnecessary sentences.
- **Caching of already downloaded pages**: for each webpage, the content is stored in a JSON file, as well as the found sublinks. *Example*: after a run with a limit of N pages, other runs with less than N pages will use the stored files instead downloading data from internet; at the contrary, if the limit is increased to M > N pages, only M - N additional pages will be downloaded while the first N pages will be taken from the stored file.

## Load the results from the crawler's repo

In [8]:
github_downloader.download_files_from_github_repo(folderName="", saveFolder="/kaggle/working/companies_info", extension="json")

/kaggle/working/companies_info/companies_info.json downloaded successfully.
All files downloaded.


Locate the JSON file containing the companies' information.

In [9]:
import os

# if the JSON file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_companies = False if os.path.exists('/kaggle/working/companies_info') else True
companies_json_name = 'companies_info.json'
if use_kaggle_input_companies:
    companies_info_file_path = os.path.join('/kaggle/input/companies-info', companies_json_name)
else:
    companies_info_file_path = os.path.join('/kaggle/working/companies_info', companies_json_name)

print(f"The file {companies_info_file_path} will be used for the information regarding the companies")

The file /kaggle/working/companies_info/companies_info.json will be used for the information regarding the companies


Define a small function to read the information about the companies - in JSON format.

In [10]:
import json

def read_json_info(jsonFilePath: str) -> dict:
    if os.path.exists(jsonFilePath):
        with open(jsonFilePath, "r") as f:
            data = json.load(f)
        return data
    else:
        return {}

Load the companies' information by using the defined function.

In [11]:
companies_info = read_json_info(companies_info_file_path)

# companies_info is a dictionary, where the key is the name of the company and the related value its information
# print(companies_info["SIEMENS"]) 

# Chat with Gemini

In [12]:
# API key got here: https://ai.google.dev/tutorials/setup

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
secret_key = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key = secret_key)

model_name = 'gemini-1.5-flash-latest'
model = genai.GenerativeModel(model_name=model_name)

chat = model.start_chat()

model_info = genai.get_model(f"models/{model_name}")
print(f"{model_info.input_token_limit=}")
print(f"{model_info.output_token_limit=}")

model_info.input_token_limit=1000000
model_info.output_token_limit=8192


## Analyze all the tenders

In [13]:
download_tenders_json_from_repo = False # switch to False if willing to generate the json file within this notebook

if download_tenders_json_from_repo:
    # this helps reducing the Gemini Quota, since no queries will be performed for the tenders
    github_downloader.download_files_from_github_repo(folderName="tenders", saveFolder=tenders_file_path, extension="json")

In [14]:
tender_prompt_template = "The document you have is a tender, that contains technical requirements for a project. Summarize the technical requirements. The content of the document is: "
tender_prompts = []
for info in tenders_info.values():
    tender_prompts.append(f"You have a document called {info['name']} . " + tender_prompt_template + f"{info['content']}")

In [15]:
from time import sleep

responses = {}
tenders_json_file_path = os.path.join(tenders_file_path, 'tenders.json')
if os.path.exists(tenders_json_file_path):
    responses = read_json_info(tenders_json_file_path)
    print(f"Responses loaded from file {tenders_json_file_path}")
else:
    num_queries = 0
    for tender_prompt, tender_name in zip(tender_prompts, tenders):
        print(f"Generating response for tender {tender_name} ...")
        response = chat.send_message(tender_prompt)
        # print(response.text)
        responses[tender_name] = {'prompt': tender_prompt, 'answer': response.text}
        print(f"Response for tender {tender_name} generated.")

        num_queries += 1
        """
        if num_queries % 2 == 0:
            wait_time_seconds = 90
            print(f"Waiting {wait_time_seconds} before continuing, to not exceed Gemini's quota")
            sleep(wait_time_seconds)
        """

    with open(tenders_json_file_path, 'w') as f:
        json.dump(responses, f, ensure_ascii=True, indent=4)
    print(f"Responses stored into {tenders_json_file_path}")

print("Analysis of the tenders concluded!")

Generating response for tender tender_solar.pdf ...
Response for tender tender_solar.pdf generated.
Generating response for tender tender_wind.pdf ...
Response for tender tender_wind.pdf generated.
Responses stored into /kaggle/working/tenders/tenders.json
Analysis of the tenders concluded!


In [16]:
# example how to include the chat history here https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb
# description of the Content class here https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md
from google.generativeai.protos import Content, Part

history_chat = []
for idx_response, response in enumerate(responses.values()):
    query = Part()
    query.text = response['prompt']
    # TODO: consider different users? Example: for the tender number idx_response, use f"user_{idx_response}"
    history_chat.append(Content(role="user", parts=[query]))

    answer = Part()
    answer.text = response['answer']
    history_chat.append(Content(role="model", parts=[answer]))

## Define the system and user prompts

In [17]:
system_prompt = "You are an experienced technical sales manager. You are given the full portfolio for products and solutions of companies, through their website. Use this information to provide detailed technical answers based on the tender requirements which were given to you."

Markdown(system_prompt)

You are an experienced technical sales manager. You are given the full portfolio for products and solutions of companies, through their website. Use this information to provide detailed technical answers based on the tender requirements which were given to you.

In [18]:
user_prompt = f"""

1. Identify all the technical requirements in the tenders' information you have. This information is in the form of text I provided, then you do not need to read additional documents.

2. For company [SIEMENS] and [HITACHI], find the respective relevant products and solutions with respect to point 1. Do this one company at a time and store the results. This information is in the form of text I provided, then you do not need to read additional documents or access to websites.

3. Calculate an affinity score in percentage of company [SIEMENS] and [HITACHI] based on the match of results in point 2. Explain the way how you computed this percentage.

SIEMENS: {companies_info["SIEMENS"]}
HITACHI: {companies_info["HITACHI"]}

4. Return a quick technical summary of the tender.

5. Return the technical details of the company with the highest affinity score and its score. Focus on the technical specifications mentioning the compliances with the tender. Report also the URL of the source where you found the informations and mention if there is some non compliant requirements from tender.

6. Return also the other affinity scores alone of the other companies. 

"""

# Markdown(user_prompt)

## Test the history

In [19]:
chat_with_memory = model.start_chat(history=history_chat)
response = chat_with_memory.send_message("What are the tenders you have about?")

Markdown(response.text)

I have information about two different types of tenders:

1. **A tender for a solar power plant:** This tender specifies the technical requirements for building a 200 kWp solar power plant, including the solar array, mounting structure, inverter, battery storage, diesel generator backup, electrical system (cables, junction boxes, etc.), powerhouse building, monitoring system, and mini-grid infrastructure (transformers, transmission/distribution lines, customer connections).  The tender also includes requirements for manuals, warranties, spare parts, training, and after-sales service.

2. **A tender for wind energy projects (more accurately, a discussion of tender *design* for wind energy):** This document does *not* describe a specific wind energy project tender. Instead, it's a report analyzing different approaches to designing effective tendering *systems* for wind energy projects, both onshore and offshore.  It discusses various design options (tender organization, site selection, pre-qualification criteria, penalties, remuneration methods, price-finding mechanisms, etc.) and provides case studies of how different countries have structured their wind energy tenders, along with their successes and failures.  The focus is on creating a system that encourages competition, drives down costs, and ensures the efficient deployment of wind energy.


In [20]:
response = chat_with_memory.send_message("Detailed explanation of the technical specifications of the tenders")

Markdown(response.text)

Let's break down the technical specifications of the two tenders separately:

**Tender 1: Solar Power Plant**

This tender outlines detailed specifications for various components of a 200 kWp solar power plant and its associated mini-grid.  The technical specifications are designed to ensure a reliable, efficient, and safe system.  Here's a breakdown by component:

* **Solar PV Array (200 kWp):**  This is the core of the system.  The specifications dictate the use of either monocrystalline or polycrystalline silicon PV modules with a power capacity of 200 kWp at Standard Test Conditions (STC).  Crucially, the modules must meet specific performance and durability criteria:  a minimum 25-year performance warranty guaranteeing at least 80% of initial power output, a maximum 1% annual degradation, and adherence to IEC 61215 (performance testing) and IEC 61730 (safety) standards.  Furthermore, the tender specifies requirements for labeling, including brand, model, nominal power, voltage, current, open-circuit voltage, and short-circuit current.  The manufacturer must meet ISO 9001:2008 (quality management) and ISO 14001:2004 (environmental management) standards.

* **PV Mounting Structure (200 kWp):**  This structure supports the solar panels.  It must be non-corrosive (likely aluminum or galvanized steel), capable of adjusting tilt angles for optimal energy production, and robust enough to withstand wind speeds up to 150 kph.  Specific material requirements, coating thickness (minimum 120 microns for galvanization), and fastener materials (stainless steel 304) are detailed.  Clearances from fences and pathways for maintenance access are also specified.

* **Inverter (140 kW):** This converts the DC power generated by the solar panels into AC power for use in the grid.  The tender demands high efficiency (>95%), low no-load losses (<1%), and a wide range of acceptable grid voltage and frequency parameters.  The inverter must have features like sinusoidal current modulation, optional VAR control, data logging capabilities (with Ethernet and/or other communication interfaces), various safety protections (overcurrent, over-temperature, etc.), and remote monitoring and control features (possibly via modem or web server).  The protection class (IP31) is specified.

* **Battery (400 kWh):** This provides energy storage and grid stabilization. Lithium-ion technology is mandatory, prioritizing high power density over energy density.  The battery must meet performance criteria including cycle life (minimum 1000 cycles at 90% depth of discharge), self-discharge rate (≤5% per month), and capacity retention (at least 80% after two years).  Quality and performance standards (ISO 9001:2008/ISO 14001:2004 and IEC 61427-1 or equivalent) are specified, as is the required information to be displayed on the product.


* **Diesel Genset (180 kW/225 kVA):**  This serves as a backup power source. Specific requirements cover engine type (four-stroke, multi-cylinder, water-cooled, turbocharged), cold-starting capability, smart starting control, monitoring and metering features, and compliance with standards like ISO 3046/BS 5514/IS 4722/1992.

* **Electrical Balance of System (BOS):** This encompasses all other electrical components, including cables, junction boxes, and wiring.  Specifications cover cable types, insulation, voltage ratings, protection methods (underground laying, conduit use), and power loss limits.  Specific details are provided for junction boxes (array, sub-main, and main), including their construction, waterproofing, and surge protection.

* **Powerhouse Buildings and Parking:**  This specifies building requirements for a control room (minimum area, features like office space, toilet, and pantry), and parking space for vehicles.

* **Monitoring System:**  A SCADA (Supervisory Control and Data Acquisition) system is mandated to monitor various parameters (temperatures, solar radiation, inverter outputs, etc.), provide remote access via a web portal, and meet IEC 61724 standards.  The tender details the specific parameters to be monitored and logged.

* **Other requirements:**  The tender also includes detailed specifications for the mini-grid infrastructure, spare parts, manuals, warranties, training, and after-sales services.


**Tender 2: Wind Energy (Tender Design)**

This is not a tender for a specific wind project but rather a framework for designing effective tenders for wind energy projects.  The "technical specifications" here are not about individual components but rather about the *process* of creating a competitive and effective tender system:

* **Pre-qualification Criteria:**  The report discusses the need for carefully balanced pre-qualification criteria to ensure the participation of credible bidders without excluding smaller players.  It distinguishes between material (permits, grid connection) and financial requirements (bid bonds, financial track record).

* **Penalties for Non-Compliance:**  This outlines the design of penalties for delays or non-completion of projects, emphasizing the need for a balance to incentivize performance without unduly deterring bids.

* **Remuneration Methods:** This explores different ways to compensate successful bidders, including paying for capacity or energy produced, each with its own implications for competition and cost-effectiveness.

* **Auction Mechanisms:**  Various auction formats are discussed, such as sealed-bid, iterative (descending or ascending clock auctions), and hybrid approaches, with their respective advantages and disadvantages.

* **Technology-Specific vs. Technology-Neutral:** The report explores the pros and cons of designing tenders that are open to all technologies versus those that focus on specific technologies.

The case studies included in this report are a form of technical specification, providing real-world examples of successful and unsuccessful approaches to tender design. The details of these national tender systems (Italy, Netherlands, Portugal, UK, Brazil, South Africa, Poland, etc.) highlight specific choices made regarding these design aspects, their impact on results, and lessons that can be applied to future tender designs.  For instance, the differences between a centralised and decentralised approach to site selection and grid connection responsibilities between the UK and Denmark's offshore wind tenders are explored.

In essence, the second "tender" is a meta-tender; a framework for creating effective tenders for wind energy projects, rather than specifications for a particular project.


## Count the needed tokens

In [21]:
print(f"{model.count_tokens(history_chat)=}")
print(f"{model.count_tokens(system_prompt)=}")
print(f"{model.count_tokens(user_prompt)=}")

model.count_tokens(history_chat)=total_tokens: 20492

model.count_tokens(system_prompt)=total_tokens: 44

model.count_tokens(user_prompt)=total_tokens: 645267



## Generate the final response

In [ ]:
# wait some time, until the Gemini's Quota comes back
sleep(60) # seconds

In [23]:
response = chat_with_memory.send_message(f"{user_prompt}")

Markdown(response.text)

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [ ]:
# wait some time, until the Gemini's Quota comes back
sleep(60) # seconds

In [ ]:
print("Providing prompts based on the previous analysis of the tenders ...")
big_prompt = f"{system_prompt}\n\n{user_prompt}"
response = chat_with_memory.send_message(big_prompt)
# print(response.text)
responses['main_query'] = {'prompt': big_prompt, 'answer': response.text}

print("Response to the prompts is ready!")

In [ ]:
Markdown(responses['main_query']['answer'])